In [1]:
%%javascript 
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

# Variational inference for a simple one parameter linear model

### Basics

VI (see [Blei et al 2018](https://arxiv.org/abs/1601.00670)) is a way of turning the approximation of a posterior distribution, $p(\theta | D)$, from a sampling problem (ie using MCMC) into an optimization problem. The advantage of doing this is that we can build an approximate distribution more quickly via optimization. The disadvantage is often a poorer approximation in many cases (specifically, we have to choose the form of the approximation and so misspecification at this stage cannot be overcome even asymptotically, unlike most sampling methods). 

VI aims to find the distribution closest to the posterior according to the Kullback-Leibler divergence
$$q^*(\theta) = \arg \min_q KL(q(\theta) || p(\theta | D))$$
We usually restrict $q$ to lie in some parametric family of distributions, ie, $q=q_\phi$, and then find
$$\phi^* = \arg \min_\phi KL(q_\phi(\theta) || p(\theta | D))$$

We can't compute this directly, but by noting that

\begin{align*}
KL(q_\phi(\theta) || p(\theta | D)) &= \mathbb{E}_{\theta \sim q} (\log q(\theta) - \log p(\theta | D))\\
&= \mathbb{E}_{\theta \sim q} \log q(\theta) - \mathbb{E}_{\theta \sim q} \log p(\theta, D) + \log p(D)
\end{align*}

we can see that minimizing the KL is equivalent to maximizing 
\begin{equation}
\mathcal{L}(\phi) = \mathbb{E} \log p(\theta, D) - \mathbb{E} \log q_\phi(\theta)
\end{equation}
where all expectations are with respect to $\theta \sim q_\phi(\theta)$. $\mathcal{L}$ is called the evidence lower bound (ELBO). 


An equivalent expression that will be useful is 
\begin{equation}
\mathcal{L} = \mathbb{E} \log p(D|\theta) - KL(q(\theta)|| p(\theta))
\end{equation}
ie, the ELBO is a trade-off between minimizing the expected log-likelihood of the data under $q$ whilst not moving too far from the prior distribution $p(\theta)$. It is a balance between fitting the data, and taking account of the prior.

### Linear model


### REWRITE THIS!!!!!!@!!!!!

Consider the simple linear model
$$y = X\beta+ e \qquad e \sim N_n(0, I)$$
with prior distribution $\beta \sim N_n(0, 100 I)$, where $X$ is a $n \times p$ design matrix, $y$ a $n$ vector of observations, and $\beta$ a p-vector of parameters to be learnt.


Can the posterior can be computed exactly in this case?

We'll now consider variational inference for this model. We will use  Gaussian  distributions as the variational family 
$$q_\phi(\beta) = N(\mu, D)$$
where $D={diag}(\sigma^2_i)$ is a diagonal matrix of variances.
We thus have $2p$ variational parameters to learn. We'll arrange these into a $p \times 2$ matrix with $i^{th}$ row $\phi_{i.}=(\mu_i, \log \sigma_i^2)$. We use $\log \sigma^2$ to avoid problems with negative variances. Note that the variational family is of the correct parametric form, so that VI should give us the exact posterior distribution in this case (ie $q^* = p(\theta|D)$ if we do everything correctly).

We want to choose $\phi$ to maximize the ELBO
$$\mathcal{L}(\phi) = \mathbb{E}_{\beta\sim q_\phi} \log p(y|X,\beta)- KL(q_\phi(\beta)||p(\beta))$$
For this model, we have
$$\log p(y|x,a) \propto -\frac{1}{2} (y-X\beta)^\top(y-X\beta) = -\frac{1}{2} (y -X\mu+X\mu -X\beta)^\top (y -X\mu+X\mu -X\beta)$$
where we've ignored terms that don't depend on the variational parameters (as they won't matter when we maximize). Expanding and taking the expectation wrt $q$, the cross-terms are zero, so we get 
$$\mathbb{E}_{\beta\sim q} \log p(y|X,\beta) = -\frac{1}{2}(y-X\mu)^\top(y-X\mu)  -\frac{1}{2}\mathbb{E}(\beta-\mu)^\top X^\top X(\beta-\mu) $$

To compute the term on the right, note that $X(\beta-\mu)\sim N(0, XDX^\top)$. We can then use the formula for the expectation of a [quadratic form](https://en.wikipedia.org/wiki/Quadratic_form_(statistics)) to see that
the expected value is $tr(XDX^\top) = tr(DX^\top X)$. Thus

$$\mathbb{E}_{\beta\sim q} \log p(y|X,\beta) = -\frac{1}{2}(y-X\mu)^\top(y-X\mu) -\frac{1}{2}tr(D X^\top X) $$


For two univariate Gaussian distributions it is easy to show
$$KL(N(\mu_1, \sigma^2_1)|| N(\mu_2, \sigma^2_2)) = -\frac{1}{2}\log \frac{\sigma_1^2}{\sigma_2^2}+\frac{\sigma_1^2+(\mu_1-\mu_2)^2}{2\sigma_2^2}-\frac{1}{2}$$
and so 
$$KL(N_p(\mu, diag(\sigma^2_i))|| N_p(0, 100I)) = -\frac{1}{2}\sum_{i=1}^p\log\sigma_i^2+\sum \frac{\sigma^2_i + \mu_i^2}{200}$$
where I have ignored terms that will disappear when we differentiate in order to maximize.

#### Analytic VI

If we differentiate $\mathcal{L}$ with respect to $mu$ and set the derivative equal to zero we find
$$X^\top(y-X\mu) - \mu/100=0$$ which gives
$$\mu^* = (X^\top X+\frac{1}{100} I)^{-1} X^\top y$$

To differentiate wrt $\sigma^2$ we have to first note that (using Einstein's summation convention)
$$\frac{\partial}{\partial \sigma^2_i}tr(DX^\top X) = \frac{\partial}{\partial \sigma^2_i} D_{ij}X_{kj}X_{ki} = X_{ki}X_{ki}$$
so 
$$\frac{\partial}{\partial \sigma^2}tr(DX^\top X) = diag(X^\top X)$$

Differentiating the ELBO wrt to $\sigma^2_i$ gives
$$-\frac{1}{2} diag(X^\top X)_i +\frac{1}{2\sigma^2_i}-\frac{1}{200}=0$$
which then gives
$$\sigma^{2*}_i = \frac{1}{diag(X^\top X)_i + 1/100}$$


The Hessian is
$$H = (\begin{array}{cc}X^\top X +\frac{1}{100}I_{p\times p} &0 \\
0&\frac{1}{2}D^{-2}\end{array})
$$

## VI in practice
This calculation required three steps that may be difficult for more realistic models.

1. Computation of $\mathbb{E}_{\theta \sim q_\phi(\theta)} \log p(D|\theta )$
2. Computation of $KL(q_\phi(\theta)||p(\theta))$
3. Optimization of the ELBO wrt variational parameters $\phi$.

Problem 2 doesn't occur as often as 1 and 3, as the prior distribution and variational family are  usually  choosen to be simple forms allowing us to compute the KL divergence analytically.

### Using jax autodiff to do the optimization 

Let's first assume that we can compute the ELBO analytically, and that the optimization (problem 3) is hard. We can use an autodiff language such as jax to help with the optimization of the ELBO. Here I'll use [jax](https://github.com/google/jax). Jax does a number of cool things in addition to autodiff, including just in time compilation which we'll make use of to speed up the code.
 

In [2]:
import jax.numpy as np
from jax import grad, jit, random, vmap, jit
from scipy.stats import norm
import numpy as onp
key = random.PRNGKey(1)

/Users/pmzrdw/opt/anaconda3/lib/python3.7/site-packages/jax/lib/xla_bridge.py:122: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


Let's create some data to use, and compute the true posterior distribution.

In [3]:
n=100
X2 = norm.rvs(size=(n,2))*2
X2=np.concatenate((np.ones(n)[:,None], X2), axis=1)


In [4]:
#import matplotlib.pyplot as plt
#from scipy.stats import norm # should I use jax random numbers?
#x = norm.rvs(size=n)*2
betatrue = np.array((1.,2.,3.))
y= X2@betatrue+ norm.rvs(size=n)
#plt.plot(x,y, '+')

# True posterior?
y.shape

(100,)

Let's first optimize using scipy's built in optimization function. This doesn't use anything other than evaluations of the ELBO (ie it doesn't use derivative information).

In [5]:
from jax import jacfwd, jacrev,  value_and_grad
def hessian(f):
    return jacfwd(jacrev(f))

## Tidy VAE code

To make the code look more like standard VAE code we can write it in terms of a encoder and decoder.


In [6]:
class VAE():
    def __init__(self, x, y, phi0=None):
        ''' 
        x must be 2-d array with one row per observation, one column per covariate. If leading column isn't a column
        of 1s then we add one.
        
        y is 1-d array with same number of rows of x
        
        phi0 is 2-d array with p rows and 2 columns. Each row correspondsto one parameter. The first column is the 
        parameter mean, the second column is the parameter log-variance.  The first row corresponds to the intercept, 
        the subsequent rows are ordered as per the columns of x.
        '''
    
        assert x.ndim==2
        assert y.ndim==1
        self.n = x.shape[0] ## number of observations
       
        assert y.size==self.n
        if np.all(x[:,0]!=1): ## append column of 1s.
            x = np.concatenate((np.ones(self.n)[:,None], x), axis=1)
        
        self.p = x.shape[1] ## number of parameters
        self.x = x
        self.y = y
        if phi0 is None:
            phi0 = np.zeros((self.p,2))  #  better to make this random
            #one row for each regression parameter. First column is mean, second is logvar
        self.encoder(phi0)
        self.form_derivs()  ## initialise the gradients of the loss
        
    def encoder(self, phi):
        ## what should this do? not currently being used.
        '''Encoder using a gaussian distribution with parameters mu and logvar'''
        self.phi=phi
        self.mu = phi[:,0] # how to do this for general dimension?
        self.logvar = phi[:,1]
        
        self.var=np.exp(self.logvar)
        self.sd=np.exp(self.logvar/2.)
        return(None) 
    
    def print(self, key=None):
        print('--------------------------------')
        print('Mean of the parameter distributions: mu =', self.mu)
        print('Var of the parameter distributions: sigma^2 =', self.var)
        print('Std deviation of the parameter distributions: sigma =', self.sd)
        print('phi =', self.phi)
        if key is not None:
            print('loss=', model.loss(self.phi, 10000, key))
        print('---------------------------------')
        
    def reparam(self, e):
        '''
        Reparametrization trick - converts an array of S x p N(0,1) rvs into an array of S x p theta values
        self.sd is a 1d array of length p
        e * self.sd multiplies each row of e by vector of std devs. Note this is element wise not matrix multiplication
        self.mu is a 1d array of length p. Adding it to the S x p matrix seesms to broadcase in the right way.
        '''
        return(self.mu+e*self.sd)
        
    
    def decoder(self, theta):
        ''' 
        model prediction given theta
        theta must be a p-vector
        '''
        assert theta.ndim ==1
        return(self.x @ theta) # model prediction is y=theta*x
    
    def loglike(self, theta):
        '''
        loglikelihood - upto proportionality ignoring variance component - given theta
        '''
        assert theta.ndim==1
        return(-0.5*np.sum((self.y-self.decoder(theta))**2))  
    
    def kld(self, phi):
        '''
        phi = (mean, log(var))
        
        kld(N(mu, var), N(0,100)) is
        -0.5 x logvar+0.5 x log(100)+(var+mu^2)/200 - 0.5
        but as we are maximizing this we don't care about terms that don't contain parameters, so we return
        -0.5 log var+(var+mu^2)/200.
        
        CHANGE to take any allow any variance?
        '''
        assert phi.ndim==1
        assert phi.size==2
        return(-0.5* phi[1]+(np.exp(phi[1]) + phi[0]**2)/200.)
        
    def loss(self, phi, S, key):
        assert phi.shape == self.phi.shape
        self.encoder(phi) #IS THIS A BAD IDEA
        print('warning - encoder run - possibly bad idea')
        kld = np.sum(vmap(self.kld)(self.phi))  
        esamples=random.normal(key, shape=(S,self.p))
        thetasamples = self.reparam(esamples)
        Eloglike = np.mean(vmap(self.loglike)(thetasamples)) # apply loglike to each row of thetasamples and take mean
        return(kld-Eloglike)
    
    def form_derivs(self):
        self.dloss = value_and_grad(self.loss)
        self.hessian=hessian(self.loss)


### COMPUTE THE TRUE DERIVS

In [7]:


#from jax.test_util import check_grads
#check_grads(model.loss, (phi, subkey), order=1)

In [54]:
class AnalyticModel():
    def __init__(self, X, y, phi0=None):
        ''' 
        x must be 2-d array with one row per observation, one column per covariate. If leading column isn't a column
        of 1s then we add one.
        
        y is 1-d array with same number of rows of x
        
        phi0 is 2-d array with p rows and 2 columns. Each row correspondsto one parameter. The first column is the 
        parameter mean, the second column is the parameter log-variance.  The first row corresponds to the intercept, 
        the subsequent rows are ordered as per the columns of x.
        '''
    
        assert X.ndim==2
        assert y.ndim==1
        self.n = X.shape[0] ## number of observations
       
        assert y.size==self.n
        if np.all(X[:,0]!=1): ## append column of 1s.
            X = np.concatenate((np.ones(self.n)[:,None], X), axis=1)
        
        self.p = X.shape[1] ## number of parameters
        self.X = X
        self.y = y
        if phi0 is None:
            phi0 = np.zeros((self.p,2))  #  better to make this random
            #one row for each regression parameter. First column is mean, second is logvar
        self.encoder(phi0)
        self.OLS()
        self.VI()
        
    def encoder(self, phi):
        '''
        Encoder takes phi values and computes variance etc
        '''
        self.phi=phi
        self.mu = phi[:,0] # how to do this for general dimension?
        self.logvar = phi[:,1]
        self.var=np.exp(self.logvar)
        self.sd=np.exp(self.logvar/2.)
        self.D = np.diag(self.var)
        return(None) 
    
    def print(self):
        print('-----------  AnalyticModel Summary---------------------')
        print('Mean of the parameter distributions: mu =', self.mu)
        print('Var of the parameter distributions: sigma^2 =', self.var)
        print('Std deviation of the parameter distributions: sigma =', self.sd)
        print('phi =', self.phi)
        print('loss=', self.expectedloss(self.phi))
        print('---------------------------------')

    def expectedloss(self, phi):
        assert phi.shape == self.phi.shape
        logvar = phi[:,1]
        mu = phi[:,0]
        D = np.diag(np.exp(logvar))
        kld = -0.5*np.sum(logvar)+1./200. * np.sum(np.exp(logvar)+mu**2)     
        Eloglike = -0.5*(self.y-self.X @ mu).T @ (self.y-self.X@ mu) - 0.5*np.trace(D @ self.X.T @ self.X)
        return(kld-Eloglike)

    def dexpectedloss(self, phi):# hand coded derivatives for comparison
        print('warning - dexpected loss not coded gradients for sigma')
        dmu = -self.X.T @(self.y[:,None]- self.X @ phi[:,0][:,None])+phi[:,0][:,None]/100.
        # I'd have to compute deriv of log sigma2 for other derivs 
        return(dmu)
        
    def hessian_analyticloss(self, phi):
        H = onp.zeros((2*self.p, 2*self.p))
        H[0:self.p, 0:self.p] = -1.*(-self.X.T@self.X - 1./100 * onp.eye(self.p))
        H[self.p:, self.p:] = 0.5* np.diag(self.var**(-2))
        return(H)
    
    def OLS(self):
        self.betahat = np.linalg.inv(self.X.T @ self.X) @ self.X.T @self.y
        #print('Least squares parameter estimate= ', betahat)
        self.cov_betahat = np.linalg.inv(self.X.T @ self.X)
        #print('marginal variance estimates = ', onp.diag(cov_betahat))
        #print('full cov matrix=',cov_betahat)
        self.RSS = np.sum((self.y-self.X@ self.betahat)**2)
        #print('Residual error=fitted sum of squares = ', )
        
    def Bayes(self):
        pass
    
    def printOLS(self):
        print('---------- OLS summary -------------')
        print('Least squares parameter estimate= ', self.betahat)
        print('marginal variance estimates = ', onp.diag(self.cov_betahat))
        print('Residual sum of squares=', self.RSS)
        print('------------------------------------')


    def VI(self):
        self.mustar = np.linalg.inv(self.X.T @ self.X+1./100. * np.eye(self.p)) @ self.X.T @ self.y
        self.sigma2star = 1./(np.diag(self.X.T @ self.X)+1./100.) 
        self.phistar = np.concatenate((self.mustar[:,None], np.log(self.sigma2star)[:,None]), axis=1)
        self.lossstar= self.expectedloss(self.phistar)
                              
                            
    def printVI(self):
        print('---------- VI summary -------------')
        print('Optimal variational estimates of mu= ', self.mustar)
        print('Optimal variational estimates of sigma^2 = ', self.sigma2star)
        print('Optimal variational loss =', self.lossstar)
         print('Optimal phi =', self.phistar)
        print('------------------------------------')



IndentationError: unexpected indent (<ipython-input-54-c25d3f0ea1c5>, line 106)

In [33]:
mod = AnalyticModel(X2, y)
mod.print()
mod.printOLS()
mod.printVI()
print(mod.hessian_analyticloss(mod.phi))
mod.print()

-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0. 0. 0.]
Var of the parameter distributions: sigma^2 = [1. 1. 1.]
Std deviation of the parameter distributions: sigma = [1. 1. 1.]
phi = [[0. 0.]
 [0. 0.]
 [0. 0.]]
loss= 2708.9182
---------------------------------
---------- OLS summary -------------
Least squares parameter estimate=  [0.9051655 1.9661201 2.9989383]
marginal variance estimates =  [0.01042778 0.00258431 0.00298595]
Residual sum of squares= 79.2222
------------------------------------
---------- VI summary -------------
Optimal variational estimates of mu=  [0.90504336 1.9660599  2.9988465 ]
Optimal variational estimates of sigma^2 =  [0.009999   0.0024832  0.00297975]
Optimal variational loss = 49.389194
------------------------------------
[[100.01000214 -39.67985916  -8.28479385   0.           0.
    0.        ]
 [-39.67985916 402.70645142   2.52183342   0.           0.
    0.        ]
 [ -8.28479385   2.52183342 335.5

In [34]:
key, subkey = random.split(key)
model = VAE(X2, y)
phi = model.phi

print(model.loss(phi, 100000, subkey))
print(mod.expectedloss(phi))

warning - encoder run - possibly bad idea
2712.4988
2708.9182


In [31]:

model = VAE(X2, y)
#print(dphi_vae(phi, subkey))
#model.form_derivs()

phi = model.phi

h=0.0001

numerical_deriv = onp.zeros(phi.shape)
S=100000
for ii in range(phi.shape[0]):
    eps=onp.zeros(phi.shape)
    eps[ii,0]=1*h
    numerical_deriv[ii,0]=(model.loss(phi+eps, S, subkey)-model.loss(phi-eps,S, subkey))/(2.*h)
    eps=onp.zeros(phi.shape)
    eps[ii,1]=1*h
    numerical_deriv[ii,1]=(model.loss(phi+eps, S, subkey)-model.loss(phi-eps,S, subkey))/(2.*h)
    

print(numerical_deriv)
print(model.dloss(phi, 1000,subkey))
#print(dphi_vae2(phi, model, subkey, 1000))
print(mod.dexpectedloss(phi))

#Gradients wrong.
#GRADIENTS from dphi_vae aren't changing with S. Something wrong

warning - encoder run - possibly bad idea
warning - encoder run - possibly bad idea
warning - encoder run - possibly bad idea
warning - encoder run - possibly bad idea
warning - encoder run - possibly bad idea
warning - encoder run - possibly bad idea
warning - encoder run - possibly bad idea
warning - encoder run - possibly bad idea
warning - encoder run - possibly bad idea
warning - encoder run - possibly bad idea
warning - encoder run - possibly bad idea
warning - encoder run - possibly bad idea
[[   15.86914062    47.60742188]
 [ -764.16015625   200.1953125 ]
 [-1008.30078125   168.45703125]]
warning - encoder run - possibly bad idea
(DeviceArray(2684.5686, dtype=float32), DeviceArray([[  13.859123,   51.51314 ],
             [-765.5476  ,  206.27069 ],
             [-994.12134 ,  148.71306 ]], dtype=float32))
[[   12.344415]
 [ -763.39557 ]
 [-1003.87024 ]]


In [35]:


print(phi, phi.shape)

print('----- check losses ------', )
print('vae loss', model.loss(phi, 10000, subkey))
print('analytic loss', mod.expectedloss(phi))

print('----- check gradients ------', )
print('jax gradients', model.dloss(phi, 10000, subkey))
print(' analytic gradients', mod.dexpectedloss(phi))

print('----- check Hessians ------', )
H = model.hessian(phi, 1000,subkey)
print(model.hessian(phi, 1000,subkey)) # note the Hessian is independent of theta. So we could just estimate 
print(mod.hessian_analyticloss(phi))
# it once with large S and then fix it???

# To do NR we'd need to flatten phi, grad and H into vectors and a matrix respectively.



[[0. 0.]
 [0. 0.]
 [0. 0.]] (3, 2)
----- check losses ------
warning - encoder run - possibly bad idea
vae loss 2715.7659
analytic loss 2708.9182
----- check gradients ------
warning - encoder run - possibly bad idea
jax gradients (DeviceArray(2715.7659, dtype=float32), DeviceArray([[   13.288932,    48.871048],
             [ -766.2276  ,   202.3207  ],
             [-1004.88794 ,   169.43141 ]], dtype=float32))
warning - dexpected loss not coded gradients for sigma
 analytic gradients [[   12.344415]
 [ -763.39557 ]
 [-1003.87024 ]]
----- check Hessians ------
warning - encoder run - possibly bad idea
warning - encoder run - possibly bad idea
[[[[ 1.00009926e+02 -1.29319489e-01]
   [-3.96798744e+01  2.33834624e-01]
   [-8.28479385e+00 -1.45824298e-01]]

  [[-1.29320741e-01  5.26735954e+01]
   [ 5.13143018e-02  6.77597940e-01]
   [ 1.07139498e-02  4.36454192e-02]]]


 [[[-3.96798744e+01  5.13141230e-02]
   [ 4.02706543e+02 -2.37310171e+00]
   [ 2.52183962e+00  4.43879403e-02]]

  [[ 2

### Optimizing the analytic ELBO

Before we train using the vae, let's do the computation with the analytic values.



In [53]:
from scipy.optimize import minimize

def niceloss(flatphi):
    phi=onp.zeros((3,2))
    phi[0,:]=flatphi[0:2]
    phi[1,:]= flatphi[2:4]
    phi[2,:]=flatphi[4:6]
    return(mod.expectedloss(phi))

phi=np.array(((1,2),(3,4),(5,6)))
phi
phi.flatten()
niceloss(phi.flatten())


optout = minimize(niceloss, phi,method='Nelder-Mead', tol=1e-6)
flatphiopt=optout.x
phiopt = onp.zeros(phi.shape)
phiopt[0,:]=flatphiopt[0:2]
phiopt[1,:]= flatphiopt[2:4]
phiopt[2,:]=flatphiopt[4:6]
print(phiopt)
mod.printVI()

[[ 0.90510395 -4.60452739]
 [ 1.9660015  -5.99703879]
 [ 2.99888843 -5.81606696]]
---------- VI summary -------------
Optimal variational estimates of mu=  [0.90504336 1.9660599  2.9988465 ]
Optimal variational estimates of sigma^2 =  [0.009999   0.0024832  0.00297975]
Optimal variational loss = 49.389194
------------------------------------


In [30]:
mod = AnalyticModel(X2, y)
dloss= grad(mod.expectedloss)

obj=1000
dobj=1000

gamma=0.001
phi=mod.phi
print('Phi0', phi)

S=5000
while dobj >10e-3:
    mod.print()
    obj_new = mod.expectedloss(phi)
    gra = dloss(phi)
    phinew = phi - gamma *  gra
    phi=phinew
    mod.encoder(phi)
    
    dobj = np.abs(obj-obj_new)
    obj=obj_new
    print('objective=',obj, ', change in objective=', dobj)
    print('--------')

phiGD=phi
print(phiGD)
print(mod.phistar)
print(mod.lossstar)


#Why dont we get the right variances?


Phi0 [[0. 0.]
 [0. 0.]
 [0. 0.]]
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0. 0. 0.]
Var of the parameter distributions: sigma^2 = [1. 1. 1.]
Std deviation of the parameter distributions: sigma = [1. 1. 1.]
phi = [[0. 0.]
 [0. 0.]
 [0. 0.]]
loss= 2708.9182
---------------------------------
objective= 2708.9182 , change in objective= 1708.9182
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [-0.01234442  0.7633954   1.0038704 ]
Var of the parameter distributions: sigma^2 = [0.9517004  0.8180325  0.84594613]
Std deviation of the parameter distributions: sigma = [0.9755513 0.9044515 0.9197533]
phi = [[-0.01234442 -0.049505  ]
 [ 0.7633954  -0.20085323]
 [ 1.0038704  -0.16729958]]
loss= 1342.4023
---------------------------------
objective= 1342.4023 , change in objective= 1366.5159
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distrib

objective= 157.56691 , change in objective= 5.211029
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.6988083 1.9394839 2.9902065]
Var of the parameter distributions: sigma^2 = [0.525402   0.21003157 0.24272823]
Std deviation of the parameter distributions: sigma = [0.72484624 0.458292   0.49267456]
phi = [[ 0.6988083 -0.6435915]
 [ 1.9394839 -1.5604974]
 [ 2.9902065 -1.4158128]]
loss= 152.83234
---------------------------------
objective= 152.83234 , change in objective= 4.7345734
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.7183078 1.9420247 2.9914644]
Var of the parameter distributions: sigma^2 = [0.512034   0.20143513 0.23315716]
Std deviation of the parameter distributions: sigma = [0.7155655  0.44881526 0.48286352]
phi = [[ 0.7183078 -0.6693643]
 [ 1.9420247 -1.6022879]
 [ 2.9914644 -1.4560425]]
loss= 148.51094
---------------------------------
objective= 

loss= 109.18428
---------------------------------
objective= 109.18428 , change in objective= 1.5187073
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.87058294 1.9616308  2.9977067 ]
Var of the parameter distributions: sigma^2 = [0.35817823 0.11927575 0.14013228]
Std deviation of the parameter distributions: sigma = [0.5984799  0.3453632  0.37434244]
phi = [[ 0.87058294 -1.0267246 ]
 [ 1.9616308  -2.1263173 ]
 [ 2.9977067  -1.9651685 ]]
loss= 107.73717
---------------------------------
objective= 107.73717 , change in objective= 1.447113
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.87384415 1.96205    2.997815  ]
Var of the parameter distributions: sigma^2 = [0.35199606 0.11650351 0.13694407]
Std deviation of the parameter distributions: sigma = [0.59329253 0.3413261  0.37005955]
phi = [[ 0.87384415 -1.0441353 ]
 [ 1.96205    -2.149834  ]
 [ 2.997815   -1.9881

objective= 91.25702 , change in objective= 0.7654495
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.89868546 1.9652427  2.9986362 ]
Var of the parameter distributions: sigma^2 = [0.27614656 0.08506601 0.10056195]
Std deviation of the parameter distributions: sigma = [0.5254965  0.2916608  0.31711504]
phi = [[ 0.89868546 -1.2868235 ]
 [ 1.9652427  -2.4643278 ]
 [ 2.9986362  -2.2969813 ]]
loss= 90.51677
---------------------------------
objective= 90.51677 , change in objective= 0.74024963
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.89928716 1.9653201  2.998656  ]
Var of the parameter distributions: sigma^2 = [0.27249575 0.0836632  0.09892872]
Std deviation of the parameter distributions: sigma = [0.5220113  0.2892459  0.31452936]
phi = [[ 0.89928716 -1.3001323 ]
 [ 1.9653201  -2.480956  ]
 [ 2.998656   -2.3133557 ]]
loss= 89.80051
-----------------------------

objective= 80.84492 , change in objective= 0.45036316
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.9039813 1.9659234 2.9988112]
Var of the parameter distributions: sigma^2 = [0.22270484 0.06542452 0.07761811]
Std deviation of the parameter distributions: sigma = [0.47191614 0.2557822  0.27860028]
phi = [[ 0.9039813 -1.501908 ]
 [ 1.9659234 -2.7268581]
 [ 2.9988112 -2.5559545]]
loss= 80.40599
---------------------------------
objective= 80.40599 , change in objective= 0.4389267
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.9040818 1.9659364 2.9988146]
Var of the parameter distributions: sigma^2 = [0.22034864 0.0646006  0.07665206]
Std deviation of the parameter distributions: sigma = [0.46941307 0.25416648 0.27686107]
phi = [[ 0.9040818 -1.5125443]
 [ 1.9659364 -2.7395315]
 [ 2.9988146 -2.5684788]]
loss= 79.978065
---------------------------------
objective= 7

...

objective= 53.538395 , change in objective= 0.015014648
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.905043  1.9660598 2.998846 ]
Var of the parameter distributions: sigma^2 = [0.05339842 0.01369112 0.01639559]
Std deviation of the parameter distributions: sigma = [0.23108098 0.11700905 0.12804526]
phi = [[ 0.905043  -2.929974 ]
 [ 1.9660598 -4.291008 ]
 [ 2.998846  -4.110743 ]]
loss= 53.523464
---------------------------------
objective= 53.523464 , change in objective= 0.014930725
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.905043  1.9660598 2.998846 ]
Var of the parameter distributions: sigma^2 = [0.05328267 0.01366025 0.01635872]
Std deviation of the parameter distributions: sigma = [0.23083039 0.11687709 0.12790121]
phi = [[ 0.905043  -2.9321442]
 [ 1.9660598 -4.293265 ]
 [ 2.998846  -4.112994 ]]
loss= 53.508614
---------------------------------
object

loss= 53.28159
---------------------------------
objective= 53.28159 , change in objective= 0.013626099
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.905043  1.9660598 2.998846 ]
Var of the parameter distributions: sigma^2 = [0.05139708 0.01315828 0.01575902]
Std deviation of the parameter distributions: sigma = [0.22670923 0.11470954 0.12553494]
phi = [[ 0.905043  -2.968174 ]
 [ 1.9660598 -4.330704 ]
 [ 2.998846  -4.1503425]]
loss= 53.268032
---------------------------------
objective= 53.268032 , change in objective= 0.013557434
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.905043  1.9660598 2.998846 ]
Var of the parameter distributions: sigma^2 = [0.05129078 0.01313002 0.01572526]
Std deviation of the parameter distributions: sigma = [0.22647469 0.11458632 0.12540041]
phi = [[ 0.905043  -2.9702442]
 [ 1.9660598 -4.332854 ]
 [ 2.998846  -4.152487 ]]
loss= 53

loss= 53.011143
---------------------------------
objective= 53.011143 , change in objective= 0.012229919
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.905043  1.9660598 2.998846 ]
Var of the parameter distributions: sigma^2 = [0.04926399 0.01259213 0.01508249]
Std deviation of the parameter distributions: sigma = [0.22195493 0.11221468 0.1228108 ]
phi = [[ 0.905043  -3.010562 ]
 [ 1.9660598 -4.374683 ]
 [ 2.998846  -4.1942205]]
loss= 52.99897
---------------------------------
objective= 52.99897 , change in objective= 0.012172699
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.905043  1.9660598 2.998846 ]
Var of the parameter distributions: sigma^2 = [0.04916736 0.01256653 0.01505189]
Std deviation of the parameter distributions: sigma = [0.22173713 0.11210053 0.12268616]
phi = [[ 0.905043  -3.0125253]
 [ 1.9660598 -4.3767185]
 [ 2.998846  -4.1962514]]
loss= 52

objective= 52.756912 , change in objective= 0.010967255
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.905043  1.9660598 2.998846 ]
Var of the parameter distributions: sigma^2 = [0.04723248 0.01205462 0.01444009]
Std deviation of the parameter distributions: sigma = [0.21733034 0.10979354 0.12016691]
phi = [[ 0.905043  -3.0526736]
 [ 1.9660598 -4.4183073]
 [ 2.998846  -4.237747 ]]
loss= 52.745995
---------------------------------
objective= 52.745995 , change in objective= 0.010917664
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.905043  1.9660598 2.998846 ]
Var of the parameter distributions: sigma^2 = [0.04714462 0.01203141 0.01441234]
Std deviation of the parameter distributions: sigma = [0.21712813 0.10968778 0.12005142]
phi = [[ 0.905043  -3.0545354]
 [ 1.9660598 -4.4202347]
 [ 2.998846  -4.2396703]]
loss= 52.735126
---------------------------------
object

objective= 52.568134 , change in objective= 0.010070801
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.905043  1.9660598 2.998846 ]
Var of the parameter distributions: sigma^2 = [0.04570583 0.01165175 0.01395851]
Std deviation of the parameter distributions: sigma = [0.2137892  0.10794326 0.11814613]
phi = [[ 0.905043  -3.0855296]
 [ 1.9660598 -4.452299 ]
 [ 2.998846  -4.271666 ]]
loss= 52.558105
---------------------------------
objective= 52.558105 , change in objective= 0.010028839
--------
-----------  AnalyticModel Summary---------------------
Mean of the parameter distributions: mu = [0.905043  1.9660598 2.998846 ]
Var of the parameter distributions: sigma^2 = [0.04562429 0.01163025 0.01393282]
Std deviation of the parameter distributions: sigma = [0.21359843 0.10784366 0.11803737]
phi = [[ 0.905043  -3.087315 ]
 [ 1.9660598 -4.4541454]
 [ 2.998846  -4.273508 ]]
loss= 52.548126
---------------------------------
object

#### Stochastic gradient descent

In [14]:
#phi=np.array((1.,1.,1.,1.))

model_GD = VAE(X2, y)
#model_GD.form_derivs()


obj=1000
dobj=1000

gamma=0.001
phi=model_GD.phi
print('Phi0=', phi)

S=5000
while dobj >10e-4:
    key, *subkeys = random.split(key, 2)
    obj_new, gra = model.dloss(phi, S, subkeys[0])
    phinew = phi - gamma*  gra
    phi=phinew
    model_GD.encoder(phi)
    dobj = np.abs(obj-obj_new)
    obj=obj_new
    print('objective=',obj, ', change in objective=', dobj)
    print('--------')

phi_GD=phi


Phi0= [[0. 0.]
 [0. 0.]
 [0. 0.]]
warning - encoder run - possibly bad idea
objective= 2710.647 , change in objective= 1710.647
--------
warning - encoder run - possibly bad idea
objective= 1329.4712 , change in objective= 1381.1758
--------
warning - encoder run - possibly bad idea
objective= 771.4256 , change in objective= 558.0456
--------
warning - encoder run - possibly bad idea
objective= 512.99927 , change in objective= 258.42633
--------
warning - encoder run - possibly bad idea
objective= 383.8517 , change in objective= 129.14755
--------
warning - encoder run - possibly bad idea
objective= 318.31693 , change in objective= 65.53479
--------
warning - encoder run - possibly bad idea
objective= 285.83356 , change in objective= 32.483368
--------
warning - encoder run - possibly bad idea
objective= 255.32863 , change in objective= 30.504929
--------
warning - encoder run - possibly bad idea
objective= 235.8587 , change in objective= 19.469925
--------
warning - encoder run - poss

warning - encoder run - possibly bad idea
objective= 76.68538 , change in objective= 0.021820068
--------
warning - encoder run - possibly bad idea
objective= 75.94961 , change in objective= 0.7357712
--------
warning - encoder run - possibly bad idea
objective= 75.27949 , change in objective= 0.67012024
--------
warning - encoder run - possibly bad idea
objective= 75.638336 , change in objective= 0.35884857
--------
warning - encoder run - possibly bad idea
objective= 75.675476 , change in objective= 0.037139893
--------
warning - encoder run - possibly bad idea
objective= 74.7843 , change in objective= 0.8911743
--------
warning - encoder run - possibly bad idea
objective= 74.13842 , change in objective= 0.64588165
--------
warning - encoder run - possibly bad idea
objective= 74.92619 , change in objective= 0.78777313
--------
warning - encoder run - possibly bad idea
objective= 74.11499 , change in objective= 0.811203
--------
warning - encoder run - possibly bad idea
objective= 74.

In [15]:
model.print(key)
mod.printVI()
mod.printOLS()

--------------------------------
Mean of the parameter distributions: mu = Traced<ConcreteArray([0.90356594 1.9651936  2.9988294 ])>with<JVPTrace(level=1/0)>
Var of the parameter distributions: sigma^2 = Traced<ConcreteArray([0.11506568 0.03111338 0.03701368])>with<JVPTrace(level=1/0)>
Std deviation of the parameter distributions: sigma = Traced<ConcreteArray([0.3392133  0.17638986 0.1923894 ])>with<JVPTrace(level=1/0)>
phi = Traced<ConcreteArray([[ 0.90356594 -2.1622522 ]
 [ 1.9651936  -3.4701173 ]
 [ 2.9988294  -3.2964678 ]])>with<JVPTrace(level=1/0)>
warning - encoder run - possibly bad idea
loss= Traced<ConcreteArray(62.126972)>with<JVPTrace(level=1/0)>
---------------------------------
---------- VI summary -------------
Optimal variational estimates of mu=  [0.90504336 1.9660599  2.9988465 ]
Optimal variational estimates of sigma^2 =  [0.009999   0.0024832  0.00297975]
Optimal variational loss = 49.389194
------------------------------------
---------- OLS summary -------------
L

In [16]:

#print(np.sum((y-X@ beta_GD)**2))





In [17]:
#TRY USING THE SAME RANDOM NUMBERS.

phi=np.array((1.,1.,1.,1.))

model_NR = VAE(x, y,S=5000, phi0=phi)
dphi_vae = value_and_grad(model_NR.loss)
H = hessian(model_NR.loss)

obj=1000
dobj=1000

while dobj >10e-2:
    key, *subkeys = random.split(key, 3)
    obj_new, gra = dphi_vae(phi, subkeys[0])
    print('gradient=', gra)
    H = H_vae(phi, subkeys[1])  #should I use the same subkey as for the gradient???
    phinew = phi - 0.2*np.linalg.inv(H) @ gra
    print('hessian = ', H)

    phi=phinew
    model_NR.encoder(phi)
    dobj = np.abs(obj-obj_new)
    obj=obj_new
    print('objective=',obj, ', change in objective=', dobj)
    print('--------')
    
    

NameError: name 'x' is not defined

In [ ]:
model.print()
phi_NR = model.phi
phi_NR

In [ ]:
phi=np.array((1.,1.,1.,1.))

model_GD = VAE(x, y,S=5000, phi0=phi)
dphi_vae = value_and_grad(model_GD.loss)



obj=1000
dobj=1000
gamma=0.01

while dobj >10e-3:
    key, *subkeys = random.split(key, 2)
    obj_new, gra = dphi_vae(phi, subkeys[0])
    print('gradient=', gra)
    phinew = phi - gamma*  gra

    phi=phinew
    model_GD.encoder(phi)
    dobj = np.abs(obj-obj_new)
    obj=obj_new
    print('objective=',obj, ', change in objective=', dobj)
    print('--------')

phi_GD=phi

In [ ]:
print('gradient descent estimate: ', phi_GD)
print('Newton Raphson estimate: ', phi_NR)

    

In [ ]:
print('Optimized values=',model_GD.mua, model_GD.vara, model_GD.mub, model_GD.varb)
print('Optimized values=',model_NR.mua, model_NR.vara, model_NR.mub, model_NR.varb)

Is it crashing because of the Newton Method?? Would SGD work?

Fitted sum of squares

In [ ]:
print(np.sum((y-X@betahat)**2))

beta_GD = np.array((model.mub, model.mua))
print(np.sum((y-X@ beta_GD)**2))



In [ ]:
print(beta_GD)
print(betahat)

### To do

- Why do the variances look wrong? Is it just you need the tolerance low enough.... they are a relatively minor contribution to ELBO. So we need to get quite close to the optima to learn them well.

- Code adam or something more sensible for the optimization.

- Code Bayes
